In [ ]:
"""
saving files as bhv2 instead of hdf5. Here code to load bhv2.
CONCLUSION:
Stopped, since decided better tactic was bhv2 --> h5, then treat as before. Have confirmed that this owrks. 
IF for some reason want to continue tactic of bhv2 --> mat --> pkl, then here are notes of where left off:
- go to ## modifications to filedata
- need to hadn write things to make the extracted dict match old filedata.
- one general problem is that everythign in old version is like {1: [], 2: [], ...} while
in new version is list [[], [], []].  See below for example of one field where fixed.
Could change to this in bulk? not sure if in bulk would work - in some cases might not wabnt to change.
"""

In [218]:
%load_ext autoreload
%autoreload 2

%cd ..

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/data1/code/python


In [219]:
from tools.utils import * 
from tools.plots import *
from tools.analy import *
from tools.calc import *
from tools.analyplot import *
from tools.preprocess import *
from tools.dayanalysis import *
from analysis.modelexpt import *
from analysis.line2 import PROBEDATfromFD

from pythonlib.drawmodel.analysis import *
from pythonlib.tools.stroketools import *

## STARTING FROM .MAT (after convert bhv2 to mat)

In [ ]:
# Load .mat
import scipy.io as sio
fname_mat = "/data2/animals/Diego/210127/210127_113923_naive6_Diego_1.mat"

In [ ]:
# dat = sio.loadmat(fname_mat, struct_as_record=False, squeeze_me=True)
dat = sio.loadmat(fname_mat)

In [ ]:
# SCRATCH, useful things:
# got from searching somthing like "how to convert mat to dict"
dat["TrialRecord"][0,0].dtype.descr

val = dat["TrialRecord"][0,0]
val["BlocksPlayed"]

val = dat["TrialRecord"][0,0]
val.dtype

### good below

In [208]:
def _check_keys( DICT):
    """
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    """
    for key in DICT:
        if isinstance(DICT[key], sio.matlab.mio5_params.mat_struct):
            DICT[key] = _todict(DICT[key])
    return DICT

def _check_if_all_matstruct(array, check_if="true"):
    """ returns True if all items are mat_struct.
    False otherwise. flip this criterion by making check_if=false."""
    
    x = [isinstance(a, sio.matlab.mio5_params.mat_struct) for a in array]
    if check_if=="true":
        return all(x)
    elif check_if=="false":
        return all([not xx for xx in x])
    else:
        assert False
    
TYPES = []
def _todict(matobj):
    """
    A recursive function which constructs from matobjects nested dictionaries
    """
    DICT = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
#         if strg=="num_finger_raises" and isinstance(elem, int):
#             print(elem)
#             print(np.array([[elem]]))
#             assert False
        if isinstance(elem, sio.matlab.mio5_params.mat_struct):
            DICT[strg] = _todict(elem)
        elif isinstance(elem, (int, float)):
            DICT[strg] = np.array([[elem]])
        elif isinstance(elem, np.ndarray) and len(elem)>0:
            if _check_if_all_matstruct(elem):
                # then children are all structure
                for i, e in enumerate(elem):
                    elem[i] = _todict(e)
                DICT[strg] = elem
            elif _check_if_all_matstruct(elem, check_if="false"):
                # then all chyildren not structs, this is data.
                DICT[strg] = np.array([elem]) # becuase the old h5 code did this.
#                 if len(elem)==1:
#                     DICT[strg] = 1000000 # becuase the old h5 code did this.
            else:
#                 print(elem)
#                 assert False, "how can have mixture of struct and data?"
                for i, e in enumerate(elem):
                    if isinstance(e, sio.matlab.mio5_params.mat_struct):
                        elem[i] = _todict(e)
                    else:
                        assert len(e)==0, "I expect that if this mixed with struct, then is empty"
                        elem[i] = e # becuase the old h5 code did this.
                DICT[strg] = elem
                
#         elif isinstance(elem, np.ndarray):
# #             doprint=False
# #             elem2 = np.copy(elem)
#             for i, e in enumerate(elem):
#                 if isinstance(e, sio.matlab.mio5_params.mat_struct):
# #                     doprint=True
#                     elem[i] = _todict(e)
#                 else:
#                     print(e)
#                     elem[i] = np.array([[e]]) # becuase the old h5 code did this.
#                     print(elem)
#                     print(i)
#                     assert False
#             DICT[strg] = elem
#             if doprint:
#                 print(elem2)
#                 print(dict[strg])
        
        else:
            TYPES.append(type(elem))
            DICT[strg] = elem
    return DICT


def loadmat(filename):
    """
    this function should be called instead of direct scipy.io .loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    """
    data = sio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)



import numpy as np
import scipy.io as sio
fname_mat = "/data2/animals/Diego/210127/210127_113923_naive6_Diego_1.mat"
dat = loadmat(fname_mat)
print("types, for those categorized as 'else'")
print(set(TYPES))

types, for those categorized as 'else'
{<class 'str'>, <class 'numpy.ndarray'>}


In [ ]:
## == VARIOUS PROCESSING NEEDED to make dat simialr to filedata. 
# STILL IN PROGRESS.

In [209]:
dat2 = {}
dat2["ML"] = {
    "TrialRecord":dat["TrialRecord"],
    "MLConfig":dat["MLConfig"]}
for k, v in dat.items():
    if "Trial" in k:
        dat2["ML"][k]=v
dat = dat2

In [210]:
animal = "Diego"
date = "210127"
expt = "naive6"
session = 1
fname = fname_mat
resave_as_dict = True

In [211]:
TrialRecord = dat["ML"]["TrialRecord"]
MLConfig = dat["ML"]["MLConfig"]

############################## EXTRACT PARAMS FOR THIS FILE
params = {}

params["pix_per_deg"] = MLConfig["PixelsPerDegree"][()]

try:
    resolution = MLConfig["Resolution"][()].decode()
except:
    resolution = MLConfig["Resolution"]

h = int(resolution[0:resolution.find(' x ')])
tmp = resolution.find(' 59 Hz')
if tmp==-1:
    tmp = resolution.find(' 75 Hz')
v = int(resolution[resolution.find(' x ')+3:tmp])
assert (h==1024 and v==768), "diff resolution?"
params["resolution"] = (h, v)

# num trials
trials = [int(key[5:]) for key in dat["ML"].keys() if (key[:5]=="Trial" and key!="TrialRecord")]
params["n_trials"] = max(trials)
try:
    params["n_trialoutcomes"] = len(TrialRecord["User"]["TrialOutcomes"])
except KeyError as error:
    params["n_trialoutcomes"] = ()

    
params["animal"]=animal
params["date"] = date
params["expt"] = expt
params["session"] = session
params["fname"] = fname

# ################## extract all trials into a dict (e.g, dict[1] = trial data)
if False:
    # converts to dict - slow
    trials = {}
    for t in range(1, params["n_trials"]+1):
        trials[t] = group2dict(dat["ML"][f"Trial{t}"])
else:
    # saves as hdf5 group, not dict - fast
    trials = {}
    for t in range(1, params["n_trials"]+1):
        trials[t] = dat["ML"][f"Trial{t}"]

########################## OUTPUT A DICT
filedata = {
    # "data":data,
    "TrialRecord":TrialRecord,
    "MLConfig":MLConfig,
    "params":params,
    "trials":trials,
    }

###### decide if convert to dict and save
if resave_as_dict:
    # 1) conver to dict
    print('Converting to dict - may take a whiel, liek a minute or two')
    # del filedata["data"]    # delete "data", is reduntant

    # if True:
    import os
    ext = os.path.splitext(fname)[1]
    if expt==".h5":
        # then is still h5, need toconver to dict.
        print("-- group2dict")
        filedata = group2dict(filedata)
    else:
        assert ext==".mat", "dont know this?"
    
    # 2) save pickle
    import os, pickle
    fname_dict = os.path.splitext(fname)[0] + ".pkl"
    filedata["params"]["fname_dict"] = fname_dict
    print(f"Saving pickle file {fname_dict}")
    with open(fname_dict, "wb") as f2:
        print("-- saving")
        pickle.dump(filedata, f2)



Converting to dict - may take a whiel, liek a minute or two
Saving pickle file /data2/animals/Diego/210127/210127_113923_naive6_Diego_1.pkl
-- saving


## modifications to filedata

### NOTE: paused, ideally can convert mat to h5, then load that. otherwise a key problem is that everythign in old version is like {1: [], 2: [], ...}. could change to this in bulk?

In [212]:
# BlockParams shouold be dict{1...}
BP = filedata["TrialRecord"]["User"]["BlockParams"]
if len(BP)<2:
    print(BP)
    assert len(BP)>1, "im not sure the follopwing is correct. need to check"

DICT = {}
for i, v in enumerate(BP):
    DICT[f"{i+1}"] = v
filedata["TrialRecord"]["User"]["BlockParams"] = DICT



In [215]:
updateFiledataParams(filedata)

one_dot


AttributeError: 'str' object has no attribute 'keys'

In [192]:
# -- TrialOutcomes should be dict{trialnum}, not list[trialnum-1]

DICT = {}
for i, v in enumerate(filedata["TrialRecord"]["User"]["TrialOutcomes"]):
    DICT[f"{i+1}"] = v
filedata["TrialRecord"]["User"]["TrialOutcomes"] = DICT